# Network Improvements (Current)
---
This section is for conflating network improvements with the network. For now it only supports adding the improvements to existing links in the data. A future improvement would be adding in features that add new connectivity. Ideally this would be done interactively and with some automation that facilities adding in new nodes and splitting links, but it might require GIS and some more involved editing.

# Adding Bicycle Facility Improvements (Future work)
Users would bring in the network geojson files into their preferred editing environment. Then they would create a new geojson with the new features in it. New features get labelled as "improvements" or "new" based on if they would only modify link attributes or add new connectivity. Improvements are joined with spatial operations. New connectivity is added using the link topology and a precision value. New connecitvity links would need to be drawn such that their vertices are placed on the network features that they "should" intersect with. A script would then split the relevant features and add in the necessary nodes and split the new links as needed.

# TODO this notebook should be put in a different section I think

In [50]:
from pathlib import Path
import geopandas as gpd
import pandas as pd
import numpy as np

import networkx as nx
from tqdm import tqdm
import pickle

from bikewaysim.paths import config
from bikewaysim.network import matching_script

# Settings

In [51]:
buffer_ft = 100
max_hausdorff_dist = 500

# Import network

In [52]:
links = gpd.read_file(config['network_fp'] / 'networks.gpkg',layer='osm_links')

# filter to roads
links = links[links['link_type']=='road']

#add street names back to osm
#add attributes back (especially the oneway column)
osm_attrs = gpd.read_file(config['network_fp'] / f"osm.gpkg",layer='raw',ignore_geometry=True)
cols_to_keep = ['osmid','highway','name']
links = pd.merge(links,osm_attrs[cols_to_keep],on='osmid')
del osm_attrs

# GDOT Study Area

In [53]:
other_name = "coa"

In [54]:
# import improvements
improvements = gpd.read_file(config['bicycle_facilities_fp']/"network_modifications.gpkg",layer=config['mod_name'])
improvements

,coa_id,coa_facilitytype,coa_name,coa_builtby,coa_onroad,coa_bothsides,coa_source,coa_yearinstalled,coa_protection,coa_notes,coa_osm_type,geometry
0,5,Contraflow Bike Lane,Porter Pl NE,N/A,On,N,Cycle Atlanta Phase 1,NaN,Low,None,bike lane,"MULTILINESTRING ((2229463.709 1369434.188, 222..."
1,12,Protected Bike Lane,Juniper St NE,N/A,On,Y,Cycle Atlanta Phase 1,NaN,High,None,cycletrack,"MULTILINESTRING ((2230749.844 1372324.236, 223..."
2,21,Bike Lane,Murphy Ave SW,N/A,On,Y,Cycle Atlanta Phase 1,NaN,Low,None,bike lane,"MULTILINESTRING ((2222329.538 1361034.107, 222..."
3,32,Buffered Bike Lane,Whitehall St SW,N/A,On,Y,Cycle Atlanta Phase 1,NaN,Medium,None,buffered bike lane,"MULTILINESTRING ((2222329.538 1361034.107, 222..."
4,40,Two-Way Cycle Track,Courtland St SE/Gilmer St SE,N/A,None,None,Cycle Atlanta Phase 1,NaN,Unknown,None,cycletrack,"MULTILINESTRING ((2229772.685 1365781.111, 223..."
...,...,...,...,...,...,...,...,...,...,...,...,...
115,632,Bike Lane,Forsyth St SW,N/A,On,Y,ATP/TSPLOST/Renew Atlanta,NaN,Low,None,bike lane,"MULTILINESTRING ((2227381.16 1364772.939, 2227..."
116,633,Protected Bike Lane,Forsyth St SW,N/A,On,Y,ATP/TSPLOST/Renew Atlanta,NaN,High,None,cycletrack,"MULTILINESTRING ((2227577.491 1365064.927, 222..."
117,635,Protected Bike Lane,Forsyth St SW,N/A,On,Y,ATP/TSPLOST/Renew Atlanta,NaN,High,None,cycletrack,"MULTILINESTRING ((2228040.369 1365791.673, 222..."
118,641,Two-Way Cycle Track,Memorial Drive Cycle Track,GDOT,On,N,None,2024.0,High,None,cycletrack,"MULTILINESTRING ((2228926.526 1363045.4, 22293..."


In [55]:
improvements = improvements[[f"{config['mod_name']}_id",f"{config['mod_name']}_name",f"{config['mod_name']}_osm_type","geometry"]]

In [56]:
# copy to prevent modification
links_buffered = links.copy()
other_source = improvements.copy()

# buffer the osm cycleways
links_buffered.geometry = links_buffered.buffer(buffer_ft)

# intersect with coa/arc (returns coa/arc linestrings)
overlap = gpd.overlay(other_source,links_buffered)

#street name check if for bike lanes / sharrows / cycletracks
#TODO use the new method for this step
overlap['name'] = overlap['name'].apply(lambda row: matching_script.remove_suffix(row))
overlap[f"{other_name}_name0"] = overlap[f"{other_name}_name"].apply(lambda row: matching_script.remove_suffix(row))
overlap['name_check'] = overlap.apply(lambda row: matching_script.name_check(row['name'],row[f"{other_name}_name0"]),axis=1)

In [57]:
# only accept matching names
overlap = overlap[(overlap['name_check']==True)]

In [58]:
overlap.drop_duplicates(inplace=True)

In [59]:
links.columns

Index(['A', 'B', 'linkid', 'oneway', 'link_type', 'osmid', 'geometry',
       'highway', 'name'],
      dtype='object')

In [60]:
### HAUSDORFF DISTANCE CHECK ###
# add osm geometry to compare against arc/coa geometry
overlap = pd.merge(overlap,links[['linkid','geometry']],on='linkid')
overlap['hausdorff_dist'] = overlap.apply(lambda row: row['geometry_x'].hausdorff_distance(row['geometry_y']),axis=1)
overlap.drop(columns=['geometry_x'],inplace=True)
overlap.rename(columns={'geometry_y':'geometry'},inplace=True)

# replace intersected geometry with the original geometry
overlap = gpd.GeoDataFrame(overlap,geometry='geometry')

In [61]:
# for remaining multi matches choose match with the lowest hausdorff distance
min_hausdorff = overlap.groupby('linkid')['hausdorff_dist'].idxmin()
overlap = overlap.loc[min_hausdorff]

## Visually, it looks like most of the improvements were properly assigned

In [62]:
improvements_buffer = improvements.copy()
improvements_buffer.geometry = improvements_buffer.buffer(200)
m = improvements_buffer.explore(f'{other_name}_osm_type')
overlap[['linkid',f'{other_name}_osm_type','coa_name','name','name_check','hausdorff_dist','geometry']].explore(color='red',m=m)#.sort_values('hausdorff_dist')

## Clean Up

In [63]:
needed_cols = [f'{other_name}_id', f'{other_name}_name', f'{other_name}_osm_type', 'linkid', 'geometry']

In [64]:
print(overlap['coa_osm_type'].unique())
# NOTE re-classify buffered bike lanes to bike lanes
overlap.loc[overlap['coa_osm_type']=='buffered bike lane','coa_osm_type'] = 'bike lane'
print(overlap['coa_osm_type'].unique())

['bike lane' 'cycletrack' 'buffered bike lane']
['bike lane' 'cycletrack']


In [65]:
# export the cleaned version of the improvements
overlap = overlap[needed_cols]
overlap.rename(columns={f'{other_name}_osm_type':'improvement'},inplace=True)
overlap.to_file(config['bicycle_facilities_fp']/"network_improvements.gpkg",layer='coa')

In [66]:
# improvements.geometry.apply(lambda x: x.type).value_counts()
# # improvements = improvements[improvements.geometry.apply(lambda x: x.type)!='MultiLineString']

In [67]:
# # TODO I believe this is just used for visualization
# from shapely.ops import Point
# improvements = overlap.copy()
# improvements = improvements[improvements['coa_id'].isin(overlap['coa_id'])]
# starts = improvements['geometry'].apply(lambda x: Point(list(x.coords)[0]))
# ends = improvements['geometry'].apply(lambda x: Point(list(x.coords)[-1]))
# starts_and_ends = gpd.GeoDataFrame({'geometry':pd.concat([starts,ends])},crs=config['projected_crs_epsg'])
# starts_and_ends.to_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='improvements_start_end')
# starts_and_ends.explore()

In [68]:
# improvements = gpd.read_file(config['bicycle_facilities_fp']/"network_modifications.gpkg",layer=config['mod_name'])

In [69]:
# improvements[improvements['coa_id'].isin(overlap['coa_id'])]

In [70]:
# Savannah Study Area 
# buffer_ft = 100
# max_hausdorff_dist = 500
# other_name = 'savannah'
# # import improvements
# improvements = gpd.read_file(config['bicycle_facilities_fp']/"network_modifications.gpkg",layer='savannah')
# # copy to prevent modification
# links_buffered = links.copy()
# other_source = improvements.copy()

# # buffer the osm cycleways
# links_buffered.geometry = links_buffered.buffer(buffer_ft)

# # intersect with coa/arc (returns coa/arc linestrings)
# overlap = gpd.overlay(other_source,links_buffered)

# #street name check if for bike lanes / sharrows / cycletracks
# overlap['name'] = overlap['name'].apply(lambda row: matching_script.remove_suffix(row))
# overlap[f"{other_name}_name"] = overlap[f"{other_name}_name"].apply(lambda row: matching_script.remove_suffix(row))
# overlap['name_check'] = overlap.apply(lambda row: matching_script.name_check(row['name'],row[f"{other_name}_name"]),axis=1)
# # only accept matching names
# overlap = overlap[(overlap['name_check']==True)]
# overlap.drop_duplicates(inplace=True)
# ### HAUSDORFF DISTANCE CHECK ###
# # add osm geometry to compare against arc/coa geometry
# overlap = pd.merge(overlap,links[['linkid','geometry']],on='linkid')
# overlap['hausdorff_dist'] = overlap.apply(lambda row: row['geometry_x'].hausdorff_distance(row['geometry_y']),axis=1)
# overlap.drop(columns=['geometry_x'],inplace=True)
# overlap.rename(columns={'geometry_y':'geometry'},inplace=True)

# # replace intersected geometry with the original geometry
# overlap = gpd.GeoDataFrame(overlap,geometry='geometry')
# # for remaining multi matches choose match with the lowest hausdorff distance
# min_hausdorff = overlap.groupby('linkid')['hausdorff_dist'].idxmin()
# overlap = overlap.loc[min_hausdorff]
# ## Visually, it looks like most of the improvements were properly assigned
# improvements_buffer = improvements.copy()
# improvements_buffer.geometry = improvements_buffer.buffer(200)
# m = improvements_buffer.explore(f'{other_name}_osm_type')
# overlap[[f'linkid',f'{other_name}_osm_type',f'{other_name}_name','name','name_check','hausdorff_dist','geometry']].explore(color='red',m=m)#.sort_values('hausdorff_dist')
# ## Clean Up
# overlap
# needed_cols = [f'{other_name}_id', , f'{other_name}_osm_type', 'linkid', 'geometry']
# # #replace
# # overlap[[f'{other_name}_osm_type','facility_fwd']].value_counts(dropna=False)
# # overlap.loc[(overlap[f'{other_name}_osm_type']=='cycletrack') & (overlap['facility_fwd']=='bike lane'),needed_cols+['facility_fwd']].explore(m=m)
# overlap = overlap[needed_cols]
# overlap.rename(columns={f'{other_name}_osm_type':'improvement'},inplace=True)
# overlap.to_file(config['bicycle_facilities_fp']/"network_improvements.gpkg",layer=f'{other_name}')
